## Important first step to every notebook

- References that we are creating our own local ishbook
- Imports all the Python libraries that we will need
    - Pandas (PANDA PANDA PANDA) is a Python Data Analysis library (short for panel data)
    - Plus is used to pulled relevant info for Indeed employees / advertisers
    - Datetime is a really useful module I will be using later (patience)

In [1]:
%reload_ext ishbook
import plus
import pandas as pd
import datetime as dt

## Enter start and end dates for query (we gotta start somewhere!)

In [2]:
start = '2016-08-01'
end = '2016-08-08'

## Pull all labeled NPS surveys from labeler queue (what does this really mean?)

- For each NPS survey (in the US) that has been filled out, it has been associated to a specific response type
    - If a client had a bad >:( user experience, there's certain criteria for those responses
    - If a client had a good :P experience, we have criteria for that too
    - Grabs the survey *dockey* (email token - WE REALLY NEED THIS), *answer* (labeled response)

In [3]:
%%ish

responses = from lblrresponse {start} {end} WHERE taskname:nps_queue_naus GROUP BY dockey, answer
output=None
responses.head(5)

,dockey,answer,
0,05bef6681f297b0a,cost effective,1
1,08df4b5608045cc3,cost effective,1
2,124484ed6777f359,cost effective,1
3,13128624b07744eb,cost effective,1
4,1e72d927b3bb61fa,cost effective,1


## Get Advertiser ids based on the dockey - We need to know who it is gave us that feedback, right?!
- Uses a plus call (the first of MANY) on **get_nps_verbatim** endpoint to get back advertiser id information
- Also grabs the ACTUAL verbatim responses for the NPS survey (why the client selected the score they did)
    - Resets the index to align the rows for merge

In [4]:
nps = plus.get_nps_verbatim(responses.dockey)
nps.reset_index(inplace=True)
nps.tail(5)

LDAP name:
dmoncada
Password:
········


,emailtoken,advertiser_id,candidates,cost,improvement,likely,likely_reason,notusing_other,notusing_reason,other,service,survey_type
2427,ffa8c518b36d3947,6077911,None,None,Love it. Pricey but good.,10,"I like so many things about it: tracking, org...",None,None,None,None,NPS
2428,ffb523d8de943ecb,5853391,None,None,None,10,None,None,hired_from_indeed,None,None,NPS_CHURN
2429,ffc46030cec7f1d3,8159304,None,None,screening,2,lousy applicants - they never called back or ...,None,None,None,None,NPS
2430,ffe09d21b537a508,8159373,None,None,Do not allow candidates to enter the informati...,10,Within days I received a lot of resumes to cho...,None,None,None,None,NPS
2431,ffe7b2e3b9b4b1da,433940,None,None,Have the candidates apply directly to our ATS,8,A lot of candidates,None,None,None,None,NPS


## Clean up NPS df and merge with Responses df - We call this data munging (which sounds weird, or is just me?)

- Create a separate df and call it **nps_cleaned** (my df names aren't very creative, are they?)
    - Only include 'advertiser_id','emailtoken','likely','survey_type' columns
- Merge the **nps_cleaned** df with **responses** df using an inner join on *emailtoken* & *dockey*
    - Just like that, we have advertiser tied to their NPS score tied to what that actual answer represents

In [5]:
nps = nps[['advertiser_id','emailtoken','likely','survey_type']]
nps_cleaned = nps.merge(responses,how='inner',left_on=['emailtoken'],right_on=['dockey'])
nps_cleaned.head(5)

,advertiser_id,emailtoken,likely,survey_type,dockey,answer,
0,7203593,0027d8218b0d20be,10,NPS,0027d8218b0d20be,quality candidate experience,1
1,748918,004dfe8c36149186,10,NPS,004dfe8c36149186,quality candidate experience,1
2,6856114,0054605627e8699f,10,NPS_CHURN,0054605627e8699f,hired from indeed,1
3,6698841,00bfc0391f1b9619,1,NPS_CHURN,00bfc0391f1b9619,hired from non-indeed,1
4,6698841,00bfc0391f1b9619,1,NPS_CHURN,00bfc0391f1b9619,skip,1


## Next step: Get the service rep information for each of the surveys - to tie back to individuals and teams

- Create a string of email tokens from the **nps_cleaned** df and assign it the *token* variable name
    - But really, what's a string?  Just a fancy name for list of **values**
- Grab this super long list of email token and plug that into the **npsemployernew** index (which is where completed NPS surveys live)
    - This is used to find the assigned service rep at the time the survey was completed
    - Unfortunately this index doesn't have a useful time stamp, so we have do that on our own (I really really hate unixtime)
    

In [6]:
tokens = ','.join(str(x) for x in nps_cleaned.emailtoken)

In [8]:
%%ish

nps_date = from npsemployernew 2016-01-01 today where emailtoken IN ({tokens}) group by emailtoken, country,service_rep_username,unixtime
output = None
nps_date.tail(5)

,emailtoken,country,service_rep_username,unixtime,
2427,765d9f7519c74195,US,jmccloskey,1470399178,1
2428,6dccacde10a9f27a,US,smbsupp,1470399262,1
2429,ab552bd4a3837090,US,smbsupp,1470399382,1
2430,1f924f89912bd62e,US,smbsupp,1470399414,1
2431,c5572ce631162791,US,sharedclientsupport,1470399475,1


## Because unixtime isn't really useful, we have to create a separate column for the date, then merge our two dfs

- Use the *datetime* function to create a *date* column from the data available in the *unixtime* column

In [9]:
nps_date['Date']=[dt.datetime.fromtimestamp(x).strftime('%Y-%m-%d') for x in nps_date.unixtime]
nps_date.tail()

,emailtoken,country,service_rep_username,unixtime,,Date
2427,765d9f7519c74195,US,jmccloskey,1470399178,1,2016-08-05
2428,6dccacde10a9f27a,US,smbsupp,1470399262,1,2016-08-05
2429,ab552bd4a3837090,US,smbsupp,1470399382,1,2016-08-05
2430,1f924f89912bd62e,US,smbsupp,1470399414,1,2016-08-05
2431,c5572ce631162791,US,sharedclientsupport,1470399475,1,2016-08-05


## Merge nps_date & nps_cleaned dfs, narrow down columns - MORE MUNGING

- Create a df called **nps_merged** using an inner join on *emailtoken* (since both of my dfs have this column, makes it really easy to combine them together)
- Create a df called **cs_list** to filter out columns needed (again, this is why variable names are **important**... I have NO CLUE what cs_list is supposed to mean)
    - But NOW I have advertiser linked to NPS score, answer, country, service rep, and the date the survey was filled, so we're making progress!

In [10]:
nps_merged = nps_cleaned.merge(nps_date,how='inner',on='emailtoken')
cs_list = nps_merged[['advertiser_id','likely','survey_type','answer','country','service_rep_username','Date']]
cs_list.tail()

,advertiser_id,likely,survey_type,answer,country,service_rep_username,Date
2884,5853391,10,NPS_CHURN,hired from indeed,US,DCASharedSupport,2016-08-04
2885,5853391,10,NPS_CHURN,skip,US,DCASharedSupport,2016-08-04
2886,8159304,2,NPS,poor candidate experience,US,smbsupp,2016-08-03
2887,8159373,10,NPS,quality candidate experience,US,DCASharedSupport,2016-08-04
2888,433940,8,NPS,misc,US,sharedclientsupport,2016-08-02


## Create an Small Business Support specific df to filter results (since my requester is only interested in his team)

- Filter on the *service_rep_username* column for **smbsupp**
    - Only include countries that the US team supports, US and CA

In [13]:
cs_list_smb = cs_list[(cs_list.service_rep_username=='smbsupp') & (cs_list.country=='US') | (cs_list.country=='CA')]
cs_list_smb.tail()

,advertiser_id,likely,survey_type,answer,country,service_rep_username,Date
2879,6198566,8,NPS,skip,US,smbsupp,2016-08-02
2880,3396292,0,NPS_CHURN,poor candidate experience,US,smbsupp,2016-08-04
2881,3396292,0,NPS_CHURN,skip,US,smbsupp,2016-08-04
2882,8170099,10,NPS,quality candidate experience,US,smbsupp,2016-08-03
2886,8159304,2,NPS,poor candidate experience,US,smbsupp,2016-08-03


## This is where I start getting fancy

- The requester only wants to see negative results for his team's clients... That's weird, why negative only?
    - He wants to go after the clients that didn't have a good experience... To see if he can get them to come around (OHHHH that's a GREAT idea)
- But how can I check to make sure I'm only getting the ones he specified?
    - There's a handy Pandas function for that (of course there is!  There's a whole book dedicated to them o_O)

In [14]:
cs_list_smb = cs_list_smb[(cs_list_smb.answer=='poor candidate experience') | (cs_list_smb.answer=='expensive') | (cs_list_smb.answer=='no hire')]
print cs_list_smb.answer.value_counts()

poor candidate experience    236
expensive                     63
no hire                       18
Name: answer, dtype: int64


## Wow you weren't kidding about getting fancy, what is going ON down there?

- As with everything in life, there's changes to every request.  Not only are we going after clients who had negative experiences, but we want to go after those that gave us a DETRACTOR (0-7) score...

- Well first of all, the column with the actual NPS score (*likely*) isn't currently set up to be read as a number... that's going to make it pretty difficult to set a filter on it
    - Therefore I have to set the whole column to **int** which is short for integer which is just a fancy name for number
- The second piece is to filter on the specific date range the requester wants, which is the week prior..
    - I'm getting so confident I do this on one line

In [15]:
cs_list_smb["likely"] = cs_list_smb["likely"].astype(int)
cs_list_smb = cs_list_smb[(cs_list_smb['Date'] >= start) & (cs_list_smb['Date'] <= end) & (cs_list_smb['likely'] <= 7)]
cs_list_smb.tail(5)

,advertiser_id,likely,survey_type,answer,country,service_rep_username,Date
2799,5752201,4,NPS_CHURN,poor candidate experience,US,smbsupp,2016-08-03
2844,596046,2,NPS_CHURN,poor candidate experience,US,smbsupp,2016-08-02
2847,8054179,3,NPS,poor candidate experience,US,smbsupp,2016-08-02
2880,3396292,0,NPS_CHURN,poor candidate experience,US,smbsupp,2016-08-04
2886,8159304,2,NPS,poor candidate experience,US,smbsupp,2016-08-03


## Whoa man, you got a lot of stuff going on in this piece, you better break this down

- Sure thing!  First of all, I need to start organizing this thing.  We got some cool data points, but let's actually make sense of it.
    - First, I want to organize it by date, the answer they gave, and their NPS score
    - Next, I'm going to clean it up yet again by only pulling a few of the columns needed for the final result
    - Some of these column names don't make much sense, do they?  I need to update some of these so they look nicer
    
- VOILA!  Looking pretty nice now.

In [16]:
cs_list_smb = cs_list_smb.sort_values(['Date', 'answer', 'likely'])
cs_list_smb = cs_list_smb[['Date','advertiser_id','likely','answer','service_rep_username']]
cs_list_smb = cs_list_smb.rename(columns={'likely': 'NPS Score','Date':'Survey Filled Date', 'service_rep_username':'Service Rep'})
cs_list_smb.head(10)

,Survey Filled Date,advertiser_id,NPS Score,answer,Service Rep
1989,2016-08-01,4050070,2,expensive,smbsupp
1656,2016-08-01,8109612,3,expensive,smbsupp
1282,2016-08-01,2010798,4,expensive,smbsupp
1545,2016-08-01,1373269,5,expensive,smbsupp
336,2016-08-01,3115322,5,no hire,smbsupp
576,2016-08-01,6470397,0,poor candidate experience,smbsupp
994,2016-08-01,7806645,0,poor candidate experience,smbsupp
1615,2016-08-01,8113403,0,poor candidate experience,smbsupp
2474,2016-08-01,7375591,0,poor candidate experience,smbsupp
349,2016-08-01,8146469,1,poor candidate experience,smbsupp


## Great man, we got some good stuff here... But what is this going to be used for?  I thought we wanted to outreach our clients to make them feel better ;)

- Oh right?  That's why we're doing this in the first place.  Well, of course, we need to get the relevant advertiser information for all of these.

- That means we get to use PLUS again!  Yay!
    - Well, we can get the client email from the **get_advertiser_info** index... but the requester wants the contact name for that client as well!
    
- Well, conveniently, this is located in another index called **get_advertiser_cont**
    - Did I say conveniently?  Because I actually meant the complete opposite of that..
    
- I create two more tables for this, and because I'm sooooo responsible, I can check my work by using **print** to show the results

In [17]:
email = plus.get_advertiser_info(cs_list_smb['advertiser_id'])['account_email'].to_frame()
contact = plus.get_advertiser_cont(cs_list_smb['advertiser_id'])['contact'].to_frame()
print email.head(1)
print contact.head(1)

                                   account_email
advertiser_id                                   
4050070        edossantos@allamericanasphalt.com
                      contact
advertiser_id                
4050070        Edna DosSantos


## Alright dude, you have to be reaching the end right?

- Yes!  Thankfully there's not much left to do at this point!

- First things first: we need to get that useful client email and contact information added to all the other cool stuff we've grab so far.
    - If you've been following along, you'll realize this needs yet ANOTHER merge (I wish I could say that by the end of this notebook I was a pro at merging... But alas, I'm still pretty terrible at them)
        - Since all the dfs have advertiser ids linking them, that's what I'll be using to match them up
- And then FINALLY, we have a completed dataframe that has all the relevant information that we need!
    - But wait, they're not in the right order!  I GUESS I should reorganize them... (remember munging? This is munging)

In [18]:
cs_list_smb = cs_list_smb.merge(email, how='left', left_on="advertiser_id", right_index=True).merge(contact,how='left', left_on='advertiser_id', right_index=True)
cs_list_smb_final = cs_list_smb[['Survey Filled Date','advertiser_id','account_email', 'contact','NPS Score', 'answer', 'Service Rep']]
cs_list_smb_final

,Survey Filled Date,advertiser_id,account_email,contact,NPS Score,answer,Service Rep
1989,2016-08-01,4050070,edossantos@allamericanasphalt.com,Edna DosSantos,2,expensive,smbsupp
1656,2016-08-01,8109612,well_t@yahoo.com,Wellington Tsai,3,expensive,smbsupp
1282,2016-08-01,2010798,airsource1llc@gmail.com,Heidi Knox,4,expensive,smbsupp
1545,2016-08-01,1373269,mhanson@spaceagesynthetics.com,Mark Hanson,5,expensive,smbsupp
336,2016-08-01,3115322,brian.pfeifer@kingseducation.com,Brian Pfeifer,5,no hire,smbsupp
576,2016-08-01,6470397,ap@scienturfic.com,Lauren Trad,0,poor candidate experience,smbsupp
994,2016-08-01,7806645,fpibid@sbcglobal.net,Alberto Mordoki,0,poor candidate experience,smbsupp
1615,2016-08-01,8113403,eurocrepescolorado@gmail.com,MEHDI MDOUARI,0,poor candidate experience,smbsupp
2474,2016-08-01,7375591,josh@va-drywall.com,Josh Haislip,0,poor candidate experience,smbsupp
349,2016-08-01,8146469,lizzy@roofstock.com,Lizzy Weiss,1,poor candidate experience,smbsupp


## So we have all this cool data ready to go, but it's stuck in this notebook... What to do?!

- Pandas to save the day yet again for us!  There is a great little function called **to_csv** we can use... BAM!

In [ ]:
cs_list_smb_final.to_csv('SMB_NPSscores_07292016.csv', index=False)